# File and setup

In [35]:
# Dependencies and Setup
import pandas as pd
import os
import csv
import numpy
import io
import nbformat

# File to Load (Remember to Change These)
schools_uploaded = "c:\\Users\\walla\\Pandas_Challenge\\schools_complete.csv"
students_uploaded = "C:\\Users\\walla\\Pandas_Challenge\\students_complete.csv"

# Read Purchasing File and store into Pandas data frame
schools_data_df = pd.read_csv(schools_uploaded, delimiter=",")
students_data_df = pd.read_csv(students_uploaded, delimiter=",")

# Combine the data into a single dataset
school_data_complete = pd.merge(students_data_df, schools_data_df, how = "left", on = ["school_name", "school_name"])

# Combine Data

In [44]:
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,passing_math,passing_reading,passing_overall
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,Pass,Fail,Fail
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,Fail,Pass,Fail
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,Fail,Pass,Fail
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,Fail,Fail,Fail
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,Pass,Pass,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130,Pass,Pass,Pass
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130,Pass,Pass,Pass
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130,Pass,Pass,Pass
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130,Pass,Pass,Pass


# Summary

In [45]:
# total number of schools
total_school = len(school_data_complete['school_name'].unique())
# total number of students
total_student = school_data_complete['student_name'].count()
# total budget
total_budget = sum(school_data_complete['budget'].unique())
# average math score
average_math_score = school_data_complete['math_score'].mean()
# average reading score
average_reading_score = school_data_complete['reading_score'].mean()
# overall passing rate
overall_passing_score = (average_math_score + average_reading_score)/2
# percentage of students passing math score (70 or greater)
passing_math_score = (school_data_complete[school_data_complete['math_score']>=70]['student_name'].count()/total_student)*100
# percentage of students passing reading score (70 or greater)
passing_reading_score = (school_data_complete[school_data_complete['reading_score']>=70]['student_name'].count()/total_student)*100

# dataframe of above results
district = {
    'Total Schools':total_school,
    'Total Student':'{:,}'.format(total_student),
    'Total Budget':'${:,.2f}'.format(total_budget),
    'Average Math Score':average_math_score,
    'Average Reading Score':average_reading_score,
    '% Passing Math':passing_math_score,
    '% Passing Reading':passing_reading_score,
    '% Overall Passing Score':[overall_passing_score],  
}

district_summery = pd.DataFrame(district)
district_summery

,Total Schools,Total Student,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Score
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


# School Summary

In [48]:
#school name dataframe
school_name_df = school_data_complete.groupby(['school_name'])

#school name variable, differentiated from column name
school_name_var = school_name_df['school_name']

#school type
#'.first()' method used to retrieve relevant data
school_type = school_name_df['type'].first()

#size method used due to groupby method
total_students_by_school = school_name_df.size()

#total students per school
students_per_school = school_data_complete['school_name'].value_counts()

#budget
total_budget_per_school = school_name_df['budget'].sum()

#school name size
school_name_size = school_name_df['size'].value_counts()

#budget per school
total_budget_schools = total_budget_per_school/total_students_by_school

#total budget per student
total_budget_per_student = total_budget_schools/students_per_school

#average math score
avg_math_score_by_school = school_name_df.math_score.mean()

#average reading score
avg_reading_score_by_school = school_name_df.reading_score.mean()

#percentage of students with a passing math score, 70 or greater, grouped by school
passing_math_by_school = school_data_complete[school_data_complete['passing_math']=='Pass'].groupby(['school_name']).size()
percent_passing_math_by_school = (passing_math_by_school/students_per_school)*100

#percentage of students with a passing reading score, 70 or greater, grouped by school
passing_reading_by_school = school_data_complete[school_data_complete['passing_reading']=='Pass'].groupby(['school_name']).size()
percent_passing_reading_by_school = (passing_reading_by_school/students_per_school)*100

#percentage of students with a passing overall score, 70 or greater, grouped by school
passing_overall_by_school = school_data_complete[school_data_complete['passing_overall']=='Pass'].groupby(['school_name']).size()
percent_passing_overall_by_school = (passing_overall_by_school/students_per_school)*100

#summary table for school data
school_summary_table_data = {'School Type': school_type,
    'Total Students': students_per_school,
    'Total School Budget': total_budget_schools,
    'Per Student Budget': total_budget_per_student,
    'Average Math Score': avg_math_score_by_school,
    'Average Reading Score': avg_reading_score_by_school,
    '% Passing Math': percent_passing_math_by_school,
    '% Passing Reading': percent_passing_reading_by_school,
    '% Overall Passing Rate': percent_passing_overall_by_school,}
school_summary_df = pd.DataFrame(school_summary_table_data)
school_summary_df = school_summary_df[
                        ["School Type", "Total Students", "Total School Budget",
                         "Per Student Budget", "Average Math Score", "Average Reading Score",
                         "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
school_summary_df.head(15)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


# Top Schools

In [49]:
#top five, sorted by overall passing rate
top_df = school_summary_df.sort_values(by ='% Overall Passing Rate', ascending=False)
top_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


# Bottom Performing Schools

In [51]:
#bottom five, sorted by overall passing rate
bottom_df = school_summary_df.sort_values(by ='% Overall Passing Rate')
bottom_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


# Math by Grade Summary

In [54]:
#variables for average math scores
avg_math_scores_ninth = school_data_complete[school_data_complete['grade']=='9th'].groupby('school_name')['math_score'].mean()
avg_math_scores_tenth = school_data_complete[school_data_complete['grade']=='10th'].groupby('school_name')['math_score'].mean()
avg_math_scores_eleventh = school_data_complete[school_data_complete['grade']=='11th'].groupby('school_name')['math_score'].mean()
avg_math_scores_twelfth = school_data_complete[school_data_complete['grade']=='12th'].groupby('school_name')['math_score'].mean()

#summary data for math scores
scores_math_data = {'Average Math Scores, 9th grade': avg_math_scores_ninth,
              'Average Math Scores, 10th grade': avg_math_scores_tenth,
              'Average Math Scores, 11th grade': avg_math_scores_eleventh,
              'Average Math Scores, 12th grade': avg_math_scores_twelfth,}
math_scores_df = pd.DataFrame(scores_math_data)
math_scores_df = math_scores_df[
                        ['Average Math Scores, 9th grade',
              'Average Math Scores, 10th grade',
              'Average Math Scores, 11th grade',
              'Average Math Scores, 12th grade',]]
math_scores_df.head(15)

,"Average Math Scores, 9th grade","Average Math Scores, 10th grade","Average Math Scores, 11th grade","Average Math Scores, 12th grade"
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading by Grade Summary

In [57]:
#variables for average math scores
avg_reading_scores_ninth = school_data_complete[school_data_complete['grade']=='9th'].groupby('school_name')['reading_score'].mean()
avg_reading_scores_tenth = school_data_complete[school_data_complete['grade']=='10th'].groupby('school_name')['reading_score'].mean()
avg_reading_scores_eleventh = school_data_complete[school_data_complete['grade']=='11th'].groupby('school_name')['reading_score'].mean()
avg_reading_scores_twelfth = school_data_complete[school_data_complete['grade']=='12th'].groupby('school_name')['reading_score'].mean()

#summary data for math scores
scores_reading_data = {'Average reading Scores, 9th grade': avg_reading_scores_ninth,
              'Average reading Scores, 10th grade': avg_reading_scores_tenth,
              'Average reading Scores, 11th grade': avg_reading_scores_eleventh,
              'Average reading Scores, 12th grade': avg_reading_scores_twelfth,}
reading_scores_df = pd.DataFrame(scores_reading_data)
reading_scores_df = reading_scores_df[
                        ['Average reading Scores, 9th grade',
              'Average reading Scores, 10th grade',
              'Average reading Scores, 11th grade',
              'Average reading Scores, 12th grade',]]
reading_scores_df.head(15)

,"Average reading Scores, 9th grade","Average reading Scores, 10th grade","Average reading Scores, 11th grade","Average reading Scores, 12th grade"
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# School Spending

In [58]:
#spending bins
spending_bins = [0, 585, 615, 645, 675]

# names for the four bins
group_names = ['<$585', '$585-615', '$615-645', '$645-675']

#summary
school_summary_spending = school_summary_df

school_summary_spending["Spending Ranges (Per Student)"] = pd.cut(school_summary_spending["Per Student Budget"], spending_bins, labels=group_names)

math_score_by_spending = school_summary_spending.groupby("Spending Ranges (Per Student)").mean()["Average Math Score"]
reading_score_by_spending = school_summary_spending.groupby("Spending Ranges (Per Student)").mean()["Average Reading Score"]
passing_math_by_spending = school_summary_spending.groupby("Spending Ranges (Per Student)").mean()["% Passing Math"]
passing_reading_by_spending = school_summary_spending.groupby("Spending Ranges (Per Student)").mean()["% Passing Reading"]
passing_overall_by_spending = school_summary_spending.groupby("Spending Ranges (Per Student)").mean()["% Overall Passing Rate"]

scores_by_spending = {"Average Math Score": math_score_by_spending, "Average Reading Score": reading_score_by_spending, 
                 "% Passing Math": passing_math_by_spending, "% Passing Reading": passing_reading_by_spending, 
                  "Overall Passing Rate": passing_overall_by_spending}
scores_by_spending_df = pd.DataFrame(scores_by_spending)
scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-615,83.599686,83.885211,94.230858,95.900287,90.216324
$615-645,79.079225,81.891436,75.668212,86.106569,66.112060
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


# School Size

In [59]:
#bins for size
size_bins = [0, 1000, 2000, 5000]

#three groups
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


school_summary_size = school_summary_df

school_summary_size["School Size"] = pd.cut(school_summary_size["Total Students"], size_bins, labels=group_names)

#variables to determine scores by school size
math_score_by_size = school_summary_size.groupby("School Size").mean()["Average Math Score"]
reading_score_by_size = school_summary_size.groupby("School Size").mean()["Average Reading Score"]
passing_math_by_size = school_summary_size.groupby("School Size").mean()["% Passing Math"]
passing_reading_by_size = school_summary_size.groupby("School Size").mean()["% Passing Reading"]
passing_overall_by_size = school_summary_size.groupby("School Size").mean()["% Overall Passing Rate"]

scores_by_size = {"Average Math Score": math_score_by_size, "Average Reading Score": reading_score_by_size, 
                 "% Passing Math": passing_math_by_size, "% Passing Reading": passing_reading_by_size, 
                  "% Overall Passing Rate": passing_overall_by_size}
scores_by_size_df = pd.DataFrame(scores_by_size)
scores_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# School Types

In [60]:
#set index to school type
school_summary_type = school_summary_df.set_index('School Type')

#variables to determine scores by school type
passing_math_by_type = school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
passing_reading_by_type = school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
math_score_by_type = school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
reading_score_by_type = school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
passing_overall_by_type = school_summary_df.groupby(["School Type"]).mean()["% Overall Passing Rate"]

#summary
school_summary_type = pd.DataFrame({"Average Math Score": math_score_by_type,
                                "Average Reading Score": reading_score_by_type,
                               "% Math Passing":passing_math_by_type,
                               "% Reading Passing":passing_reading_by_type,
                               "% Overall Passing Rate":passing_overall_by_type})

school_summary_type

,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


# Observable Trends

In [67]:
print ("Here are three observations based on received data")
()
print ("1) Around 90% of charter school students recieve passing grades in both math and reading.  Each subjects average grade is approximately 80%.")
()
print ("2) Percentage of students passing may depend on the schools size. For large schools (2,000 - 5,000 student), under 60% of the students received passing grades in both subjects.")
()
print ("3) However, the less spreading per student receives a higher passing %.  Spending less than $615 per student received passing rates of approximately 90%. Spending over that amount has passing rates at 66% and below,")

Here are three observations based on received data
1) Around 90% of charter school students recieve passing grades in both math and reading.  Each subjects average grade is approximately 80%.
2) Percentage of students passing may depend on the schools size. For large schools (2,000 - 5,000 student), under 60% of the students received passing grades in both subjects.
3) However, the less spreading per student receives a higher passing %.  Spending less than $615 per student received passing rates of approximately 90%. Spending over that amount has passing rates at 66% and below,
